In [ ]:
import olca_ipc as ipc
import olca_schema as schema
import pandas as pd
client = ipc.Client(8080)
client

In [ ]:
archivo_excel = "practica.xlsx" 
df = pd.read_excel(archivo_excel)

In [ ]:
# Obtener la propiedad de flujo desde la base de datos
flow_property_ref = client.get(schema.FlowProperty, name= "Number of items")

# Crear el FlowPropertyFactor
flow_property_factor = schema.FlowPropertyFactor(
        flow_property=flow_property_ref,
        conversion_factor=1.0,
        is_ref_flow_property=True
    )

    # Crear el flujo
material_flow = schema.Flow()
material_flow.name = "Material_flow"
#material_flow.description = 
material_flow.flow_type = schema.FlowType["PRODUCT_FLOW"]
material_flow.flow_properties = [flow_property_factor]  # ✅ Asegurar que se asigna correctamente
material_flow.category = "Practica 2"

# Guardar el flujo en la base de datos
client.put(material_flow)

print(f"✅ Flujo '{material_flow.name}'")

In [ ]:
# Crear el proceso
_process = schema.Process()
_process.name = "Material_Process"
#_process.description = "Proceso generado automáticamente"
_process.process_type = schema.ProcessType.UNIT_PROCESS
_process.category = "Practica 2"
_process.exchanges = []

# Crear el intercambio (flujo)
exchange = schema.Exchange()
exchange.is_input = False  # Determina si es input o output
exchange.flow = client.get(schema.Flow, name="Material_flow")  # Obtener el flujo
exchange.flow_property = schema.Ref(name="Material_flow")  # Asignar FlowProperty
exchange.amount = 1  # Definir la cantidad desde el DataFrame
exchange.is_quantitative_reference = not False # Output como referencia

# Agregar el flujo al proceso
_process.exchanges.append(exchange)

# Guardar el proceso en la base de datos
client.put(_process)

print(f"✅ Proceso '{_process.name}' creado exitosamente")

In [ ]:
df_p = pd.read_excel(archivo_excel)
df_p_filtrado = df_p.iloc[:4, 0:8]

# Mostrar los datos filtrados
print(df_p_filtrado)

In [ ]:
for index, row in df_p_filtrado.iterrows():

    if pd.isna(row["name"]):
        continue  # Saltar filas sin nombre de proceso

    # Obtener el proceso existente
    _process = client.get(schema.Process, name="Material_Process")
    
    # Obtener la unidad correspondiente
    unit_group = client.get(schema.UnitGroup, name="Units of mass")
    gram_unit = next((u for u in unit_group.units if u.name == row["unit"]), None)

        # Crear el intercambio (flujo de entrada)
    exchange = schema.Exchange()
    exchange.is_input = True # Es un input
    exchange.flow = client.get(schema.Flow, uid=row["uuid"])  # Obtener el flujo
    exchange.flow_property = schema.Ref(id=row["uuid"])  # Asignar FlowProperty
    exchange.amount = row["amount"]  # Definir la cantidad desde el DataFrame
    exchange.is_quantitative_reference = False  # Input no es referencia
    exchange.unit = gram_unit  # Asignar la unidad recuperada

    # Verificar si el flujo ya está en el proceso para evitar duplicados
    existing_flows = [ex.flow.name for ex in _process.exchanges]
    if row["uuid"] not in existing_flows:
        _process.exchanges.append(exchange)
    else:
        print(f"🔄 El flujo '{row['Flow']}' ya está en el proceso '{_process.name}', se omite duplicado.")

    # Guardar el proceso actualizado en la base de datos
    client.put(_process)

    print(f"✅ Input: '{exchange.flow}' agregado al proceso '{_process.name}'")

In [ ]:
process_ref = client.get_descriptor(schema.Process, name='Material_Process')
config = schema.LinkingConfig(prefer_unit_processes=True, provider_linking=schema.ProviderLinking.PREFER_DEFAULTS)
product_system_ref = client.create_product_system(process_ref, config)
product_system_ref

impact_method_ref = client.get_descriptor(schema.ImpactMethod, '2e5cd15d-d539-3141-a950-56d75df9d579')
setup = schema.CalculationSetup(target=product_system_ref, impact_method=impact_method_ref)
result = client.calculate(setup)

state = result.wait_until_ready()
print(f"result id: {state.id}")

for impact in result.get_total_impacts():
	print(f"{impact.impact_category.name}: {impact.amount} {impact.impact_category.ref_unit}")